In [ ]:
import gensim, nltk, os

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

results_dict = dict()
# contains input corpus

# iterate through all files in the corpus folder
for root, dirs, files in os.walk('British'):
    for file_name in files:
        print(file_name)
        with open(os.path.join(root, file_name),encoding='utf8') as rf:
            text = rf.read()
            token_list = pos_tag(word_tokenize(text))
            all_tokens = [lis[0] for lis in token_list]
            all_pos_tags = [lis[1] for lis in token_list]
                
        results_dict[file_name] = {"all_tokens": all_tokens, "all_pos_tags": all_pos_tags}

In [ ]:
import json

#with open("results_dict_British.json", "w", encoding='utf-8') as f:
    #json.dump(results_dict, f)

with open("results_dict_British.json", "r", encoding='utf-8') as f:
     results_dict = json.load(f)

In [2]:
for file_name in results_dict:
    all_tokens, all_pos_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_pos_tags']
    noun_tokens = []
    for i in range(len(all_pos_tags)):
        if all_pos_tags[i].startswith("N"):
            noun_tokens.append(all_tokens[i])
    results_dict[file_name]["noun_tokens"] = noun_tokens

In [30]:
stopwords_file = 'stopwords2.txt'

with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().split())

for filename in results_dict:
    noun_token_list = results_dict[filename]['noun_tokens']
    all_words = [word for word in noun_token_list if len(word)>=2 and word not in stopwords]
    results_dict[filename]['cleaned_noun_tokens'] = all_words
#     print(all_words)
#     break

In [31]:
def split_into_blocks(text, segment_size=150):
    # split text into a list of segments
    segment_list = []
    i = -1
    while True:
        i += 1
        left, right = i*segment_size, (i+1)*segment_size
        segment = text[left: right]
        segment_list.append(segment)
        if right >= len(text):
            break
    return segment_list

In [32]:
texts = []
numbers = []
for item in results_dict.values():
    texts.append(item['cleaned_noun_tokens'])
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)
print(len(numbers))

[3194, 3316, 3416, 5332, 8721, 10135, 10388, 10571, 11178, 11481, 13052, 13626, 14147, 14378, 15018, 15395, 15619, 16247, 16420, 17339, 17476, 18033, 18114, 18425, 18532, 18879, 18941, 20563, 21475, 22658, 23556, 27029]
32


In [33]:
texts = []
numbers = []
textid2name = dict()
for key, item in results_dict.items():
    texts.append(item['cleaned_noun_tokens'])
    textid2name[len(texts)-1] = key
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)

def split_list(l, length=800):
    target = len(l)//length + 1
    segment_length = round(len(l)/target)
    return split_into_blocks(l, segment_length)
new_texts = []

chunk2text = dict()
for i in range(len(texts)):
    text = texts[i]
    splitted_text = split_list(text)
    for j in range(len(splitted_text)):
        new_texts.append(splitted_text[j])
        chunk2text[len(new_texts)-1] = {"id of original doc": i, "name of original doc": textid2name[i], "chunk index": j, "total # chunks": len(splitted_text)}
print(len(new_texts))
texts = new_texts

[3194, 3316, 3416, 5332, 8721, 10135, 10388, 10571, 11178, 11481, 13052, 13626, 14147, 14378, 15018, 15395, 15619, 16247, 16420, 17339, 17476, 18033, 18114, 18425, 18532, 18879, 18941, 20563, 21475, 22658, 23556, 27029]
621


In [34]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [35]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(abc) for abc in texts]

In [36]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics= 6, passes=100,random_state = 1, iterations=300, chunksize = 2200,eval_every = None)
topic_list=lda.print_topics()
print(topic_list)
 
for i in lda.get_document_topics(corpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    print(i[bz][0])

[(0, '0.013*"time" + 0.009*"day" + 0.009*"ship" + 0.008*"thing" + 0.008*"men" + 0.008*"way" + 0.007*"part" + 0.007*"island" + 0.007*"sea" + 0.007*"place"'), (1, '0.020*"mr" + 0.010*"time" + 0.009*"man" + 0.009*"jew" + 0.009*"gentleman" + 0.008*"boy" + 0.008*"hand" + 0.007*"head" + 0.006*"lady" + 0.006*"door"'), (2, '0.015*"man" + 0.009*"hand" + 0.009*"time" + 0.008*"men" + 0.008*"people" + 0.007*"place" + 0.007*"woman" + 0.007*"life" + 0.006*"day" + 0.006*"way"'), (3, '0.011*"mr" + 0.010*"time" + 0.008*"room" + 0.008*"hand" + 0.008*"way" + 0.007*"man" + 0.007*"thing" + 0.006*"eye" + 0.005*"life" + 0.005*"face"'), (4, '0.012*"man" + 0.012*"hand" + 0.011*"men" + 0.009*"heart" + 0.009*"woman" + 0.009*"face" + 0.007*"death" + 0.007*"eye" + 0.007*"day" + 0.006*"word"'), (5, '0.012*"time" + 0.012*"man" + 0.009*"men" + 0.009*"way" + 0.008*"hand" + 0.008*"captain" + 0.007*"boat" + 0.006*"ship" + 0.005*"day" + 0.005*"board"')]
5
5
5
5
5
5
5
5
5
5
5
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
4
4
4
2
3


In [37]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.069986 -0.040680       1        1  27.435943
4     -0.126675 -0.091498       2        1  24.897686
5      0.003490  0.068773       3        1  17.994712
0     -0.039883  0.140622       4        1  13.952810
3      0.078197 -0.037678       5        1  10.882823
1      0.154857 -0.039539       6        1   4.836026, topic_info=         Term         Freq        Total Category  logprob  loglift
280        mr  1470.000000  1470.000000  Default  30.0000  30.0000
843      ship  1228.000000  1228.000000  Default  29.0000  29.0000
364      room   968.000000   968.000000  Default  28.0000  28.0000
549      boat  1107.000000  1107.000000  Default  27.0000  27.0000
432      time  4525.000000  4525.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
256    manner    58.500623   485.053123   Topic6  -5.9680   0.9139
2374    voice    76.063546  1584.400427   Topic6  -5.7055  -0.0073
480      word    72.444317  1996.874014   Topic6  -5.7542  -0.2874
3188    woman    72.160416  2355.477592   Topic6  -5.7582  -0.4565
277   morning    62.565466   939.225538   Topic6  -5.9008   0.3202

[503 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
4819       1  0.051080  abundance
4819       3  0.051080  abundance
4819       4  0.868368  abundance
4819       5  0.025540  abundance
26009      1  0.963879        aca
...      ...       ...        ...
3195       1  0.997780       zulu
3433       1  0.990003   zululand
3434       1  0.983083      zulus
3196       1  0.980626    zuvendi
3837       1  0.956898   zuvendis

[1122 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 6, 1, 4, 2])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
# Assuming vis_data is already prepared
# vis_data = gensimvis.prepare(lda_model, corpus, id2word)

# Save to a file
pyLDAvis.save_html(vis, 'lda_result_British.html')

In [38]:
from collections import defaultdict
import heapq






In [39]:
# Dictionary to store the top 5 contributing documents for each topic
top_contrib_docs = defaultdict(list)  # Stores lists of (probability, document_index)

# Finding the top 5 documents that contribute most to each topic
for doc_idx, doc_topics in enumerate(lda.get_document_topics(corpus)):
    for topic_id, prob in doc_topics:
        if len(top_contrib_docs[topic_id]) < 20:
            heapq.heappush(top_contrib_docs[topic_id], (prob, doc_idx))
        else:
            heapq.heappushpop(top_contrib_docs[topic_id], (prob, doc_idx))

# Sorting and printing the top 5 documents for each topic
print("Top 5 documents that contribute the most to each topic:")
for topic_id, docs in top_contrib_docs.items():
    # Sort the documents by probability in descending order
    sorted_docs = sorted(docs, key=lambda x: -x[0])
    print(f"Topic {topic_id}:")
    for prob, doc_idx in sorted_docs:
        print(f"  Document {doc_idx} with a probability of {prob}")



Top 5 documents that contribute the most to each topic:
Topic 3:
  Document 477 with a probability of 0.9989309906959534
  Document 479 with a probability of 0.9989303350448608
  Document 480 with a probability of 0.9989303350448608
  Document 481 with a probability of 0.9989303350448608
  Document 483 with a probability of 0.998930037021637
  Document 486 with a probability of 0.9989297389984131
  Document 482 with a probability of 0.9989286065101624
  Document 496 with a probability of 0.9989043474197388
  Document 510 with a probability of 0.9988982081413269
  Document 296 with a probability of 0.998887300491333
  Document 300 with a probability of 0.9988844394683838
  Document 298 with a probability of 0.9900934100151062
  Document 497 with a probability of 0.9731510281562805
  Document 476 with a probability of 0.9400413632392883
  Document 299 with a probability of 0.9306754469871521
  Document 488 with a probability of 0.9225452542304993
  Document 295 with a probability of 0.91

In [41]:
top_contrib_docs = defaultdict(list)  # Stores lists of (probability, document_index)

# Finding the top 5 documents that contribute most to each topic
for doc_idx, doc_topics in enumerate(lda.get_document_topics(corpus)):
    for topic_id, prob in doc_topics:
        if len(top_contrib_docs[topic_id]) < 50:
            heapq.heappush(top_contrib_docs[topic_id], (prob, doc_idx))
        else:
            heapq.heappushpop(top_contrib_docs[topic_id], (prob, doc_idx))

# Sorting and printing the top 5 documents for each topic
for topic_id, docs in top_contrib_docs.items():
    # Sort the documents by probability in descending order
    sorted_docs = sorted(docs, key=lambda x: -x[0])
    if topic_id ==2:
        
        for prob, doc_idx in sorted_docs:
            print(f"  Document {doc_idx} with a probability of {prob}")
            print(chunk2text[doc_idx])
        

        


  Document 84 with a probability of 0.998947024345398
{'id of original doc': 4, 'name of original doc': 'Benita.txt', 'chunk index': 12, 'total # chunks': 15}
  Document 414 with a probability of 0.9989331960678101
{'id of original doc': 20, 'name of original doc': 'She.txt', 'chunk index': 9, 'total # chunks': 23}
  Document 413 with a probability of 0.9989327788352966
{'id of original doc': 20, 'name of original doc': 'She.txt', 'chunk index': 8, 'total # chunks': 23}
  Document 426 with a probability of 0.9989324808120728
{'id of original doc': 20, 'name of original doc': 'She.txt', 'chunk index': 21, 'total # chunks': 23}
  Document 423 with a probability of 0.9989311099052429
{'id of original doc': 20, 'name of original doc': 'She.txt', 'chunk index': 18, 'total # chunks': 23}
  Document 420 with a probability of 0.9989305734634399
{'id of original doc': 20, 'name of original doc': 'She.txt', 'chunk index': 15, 'total # chunks': 23}
  Document 462 with a probability of 0.998929560

In [ ]:
# Specify the document index you want to analyze
doc_index = 0  # Replace with the desired document index

# Get the topic probabilities for the specified document
doc_topics = lda.get_document_topics(corpus[doc_index])

# Print the contributions of the specified document to different topics
print(f"Topic contributions for document {doc_index}:")
for topic_id, prob in doc_topics:
    print(f"  Topic {topic_id}: Probability {prob}")


In [63]:
def trace_chunk(chunk_id):
    """
    Trace a chunk ID back to its original tokens and document information.

    Parameters:
    - chunk_id (int): The ID of the chunk to trace.

    Returns:
    - dict: A dictionary containing the original tokens and document metadata.
    """
    if chunk_id in chunk2text:
        # Retrieve the metadata for the chunk
        chunk_info = chunk2text[chunk_id]
        original_doc_id = chunk_info["id of original doc"]
        chunk_index = chunk_info["chunk index"]
        total_chunks = chunk_info["total # chunks"]
        doc_name = chunk_info["name of original doc"]

        # Retrieve the tokens for the chunk
        tokens = texts[chunk_id]  # Since texts has been updated to new_texts (chunks)

        # Return a dictionary with the information
        return {
            "chunk_id": chunk_id,
            "original_doc_id": original_doc_id,
            "doc_name": doc_name,
            "chunk_index": chunk_index,
            "total_chunks": total_chunks,
            "tokens": tokens
        }
    else:
        return {"error": "Chunk ID not found."}

# Example usage:
chunk_id_to_trace = 416  # Replace with the chunk ID you want to trace
result = trace_chunk(chunk_id_to_trace)
print(result)

{'chunk_id': 416, 'original_doc_id': 20, 'doc_name': 'She.txt', 'chunk_index': 11, 'total_chunks': 23, 'tokens': ['half', 'world', 'empire', 'world', 'wolves', 'sense', 'godsgreedy', 'gain', 'face', 'messiah', 'spirit', 'god', 'power', 'people', 'jehovah', 'vessel', 'baal', 'vessel', 'vessel', 'god', 'people', 'brought', 'wealth', 'power', 'messiah', 'guiseand', 'remember', 'prophet', 'gothey', 'heart', 'jews', 'eye', 'world', 'place', 'people', 'wisdom', 'jerusalem', 'temple', 'rabbi', 'people', 'day', 'move', 'wrapping', 'arm', 'scar', 'milky', 'beauty', 'year', 'earth', 'jewish', 'messiah', 'hung', 'cross', 'golgotha', 'philosophy', 'jew', 'woman', 'woman', 'years', 'couch', 'eye', 'heart', 'oh', 'man', 'speaking', 'thing', 'earth', 'thing', 'dies', 'thing', 'death', 'thing', 'change', 'sculpture', 'rocky', 'wall', 'time', 'year', 'race', 'picture', 'pestilence', 'spirit', 'drawn', 'hour', 'surety', 'eye', 'world', 'time', 'world', 'born', 'stranger', 'namei', 'born', 'surety', 'hit

In [ ]:
chunk2text[66]

In [43]:
top_contrib_docs = defaultdict(list)  # Stores lists of (probability, document_index)

# Finding the top 5 documents that contribute most to each topic
for doc_idx, doc_topics in enumerate(lda.get_document_topics(corpus)):
    for topic_id, prob in doc_topics:
        if len(top_contrib_docs[topic_id]) < 200:
            heapq.heappush(top_contrib_docs[topic_id], (prob, doc_idx))
        else:
            heapq.heappushpop(top_contrib_docs[topic_id], (prob, doc_idx))

# Sorting and printing the top 5 documents for each topic
for topic_id, docs in top_contrib_docs.items():
    # Sort the documents by probability in descending order
    sorted_docs = sorted(docs, key=lambda x: -x[0])
    if topic_id ==4:
        
        for prob, doc_idx in sorted_docs:
            print(f"  Document {doc_idx} with a probability of {prob}")
            print(chunk2text[doc_idx])

  Document 163 with a probability of 0.9989444613456726
{'id of original doc': 8, 'name of original doc': 'Cleopatra.txt', 'chunk index': 21, 'total # chunks': 23}
  Document 162 with a probability of 0.9989438652992249
{'id of original doc': 8, 'name of original doc': 'Cleopatra.txt', 'chunk index': 20, 'total # chunks': 23}
  Document 152 with a probability of 0.9989427328109741
{'id of original doc': 8, 'name of original doc': 'Cleopatra.txt', 'chunk index': 10, 'total # chunks': 23}
  Document 146 with a probability of 0.9989422559738159
{'id of original doc': 8, 'name of original doc': 'Cleopatra.txt', 'chunk index': 4, 'total # chunks': 23}
  Document 155 with a probability of 0.9989417791366577
{'id of original doc': 8, 'name of original doc': 'Cleopatra.txt', 'chunk index': 13, 'total # chunks': 23}
  Document 145 with a probability of 0.9989416599273682
{'id of original doc': 8, 'name of original doc': 'Cleopatra.txt', 'chunk index': 3, 'total # chunks': 23}
  Document 143 wit

In [64]:
# Specify the document index you want to analyze
doc_idx = 416  # Replace with the desired document index

# Get the topic probabilities for the specified document
doc_topics = lda.get_document_topics(corpus[doc_idx])

# Print the contributions of the specified document to different topics
print(f"Topic contributions for document {doc_idx}:")
for topic_id, prob in doc_topics:
    print(f"  Topic {topic_id}: Probability {prob}")


Topic contributions for document 416:
  Topic 2: Probability 0.9083883762359619
  Topic 4: Probability 0.09075655043125153


In [74]:
# Specify the document index you want to analyze
doc_idx = 29  # Replace with the desired document index

# Get the topic probabilities for the specified document
doc_topics = lda.get_document_topics(corpus[doc_idx])

# Print the contributions of the specified document to different topics
print(f"Topic contributions for document {doc_idx}:")
for topic_id, prob in doc_topics:
    print(f"  Topic {topic_id}: Probability {prob}")

Topic contributions for document 29:
  Topic 2: Probability 0.3427259624004364
  Topic 4: Probability 0.4551624059677124
  Topic 5: Probability 0.2014709711074829


In [73]:
def trace_chunk(chunk_id):
    """
    Trace a chunk ID back to its original tokens and document information.

    Parameters:
    - chunk_id (int): The ID of the chunk to trace.

    Returns:
    - dict: A dictionary containing the original tokens and document metadata.
    """
    if chunk_id in chunk2text:
        # Retrieve the metadata for the chunk
        chunk_info = chunk2text[chunk_id]
        original_doc_id = chunk_info["id of original doc"]
        chunk_index = chunk_info["chunk index"]
        total_chunks = chunk_info["total # chunks"]
        doc_name = chunk_info["name of original doc"]

        # Retrieve the tokens for the chunk
        tokens = texts[chunk_id]  # Since texts has been updated to new_texts (chunks)

        # Return a dictionary with the information
        return {
            "chunk_id": chunk_id,
            "original_doc_id": original_doc_id,
            "doc_name": doc_name,
            "chunk_index": chunk_index,
            "total_chunks": total_chunks,
            "tokens": tokens
        }
    else:
        return {"error": "Chunk ID not found."}

# Example usage:
chunk_id_to_trace = 29  # Replace with the chunk ID you want to trace
result = trace_chunk(chunk_id_to_trace)
print(result)

{'chunk_id': 29, 'original_doc_id': 1, 'doc_name': 'Allan Quatermain.txt', 'chunk_index': 18, 'total_chunks': 23, 'tokens': ['ours', 'fire', 'country', 'thirty', 'side', 'marstuna', 'ninety', 'odd', 'milosis', 'road', 'half', 'width', 'forestclad', 'hill', 'lofty', 'road', 'army', 'cross', 'perception', 'woman', 'amount', 'laid', 'finger', 'neck', 'ground', 'husband', 'air', 'confidence', 'toss', 'head', 'meet', 'armies', 'spot', 'dust', 'storm', 'grave', 'xx', 'battle', 'morning', 'incident', 'map', 'sir', 'exception', 'guard', 'host', 'night', 'city', 'silent', 'garrison', 'exception', 'guard', 'men', 'cause', 'proceed', 'army', 'milosis', 'enemy', 'front', 'matter', 'city', 'horse', 'daylight', 'zuvendis', 'face', 'bore', 'trace', 'eye', 'trial', 'gate', 'horse', 'bade', 'day', 'officer', 'army', 'speaking', 'words', 'confidence', 'valour', 'victory', 'heart', 'rank', 'ground', 'today', 'mood', 'fare', 'trust', 'wits', 'people', 'duty', 'horror', 'head', 'fare', 'lord', 'victory', '